In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten,Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
import sklearn.cross_validation as valid
import sklearn.metrics as metrics
from keras.utils.visualize_util import plot

Using Theano backend.


In [2]:
# set parameters:
batch_size = 324
word_vocub_len = 5000
word_maxlen= 100
char_vocub_len = 50
batch_size = 32
word_embedding_dims = 200
nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [3]:
#build a model

#first add a word-level convolutional model
word_cnn = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
word_cnn.add(Embedding(word_vocub_len, word_embedding_dims, input_length=word_maxlen))
word_cnn.add(Dropout(0.25))

# add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
word_cnn.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use standard max pooling (halving the output of the previous layer):
word_cnn.add(MaxPooling1D(pool_length=2))
# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
word_cnn.add(Flatten())
word_cnn.add(Dense(250))
word_cnn.add(Dropout(0.5))

#then add char-level convolutional model
char_cnn = Sequential()
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
char_cnn.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1,
                           input_shape=(char_vocub_len,1)))
# we use standard max pooling (halving the output of the previous layer):
char_cnn.add(MaxPooling1D(pool_length=3))

# We flatten the output of the conv layer,
# so that we can add a vanilla dense layer:
char_cnn.add(Flatten())
char_cnn.add(Dense(250))
char_cnn.add(Dropout(0.5))

#then merge these layer into one model with several classification layers
model = Sequential()
model.add(Merge([word_cnn, char_cnn], mode='sum'))
# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.25))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))
print('start compile')
model.compile(loss='binary_crossentropy',
              optimizer='adadelta')
print('model compilled')

start compile
model compilled


In [ ]:
#further lets prepare data